# What You Can Learn from the Power BI Activity Log and REST APIs 

This set of PowerShell scripts is from a community presentation focused on the<span style="font-family:Calibri;font-size:11pt;">Power BI Activity Log and the Power BI REST APIs. These scripts are focused on understanding the kind of data which is available for understanding usage patterns and activities in the Power BI Service, for the purpose of</span><span style="font-family:Calibri;font-size:11pt;">Power BI governance, security, management, and adoption efforts.</span>The primary emphasis is awareness of the data which is available, more so than the exact PowerShell syntax.

**Accompanying presentation materials**:[https://www.coatesdatastrategies.com/presentations/#What-You-Can-Learn-From-Power-BI-Activity-Log](https://www.coatesdatastrategies.com/presentations/#What-You-Can-Learn-From-Power-BI-Activity-Log)

**Author**: Melissa Coates

**Last updated**: Feb 2, 2021

**Last tested with**: PowerShell 7.0.3 on Windows 10

**Code status**: These scripts are considered demo snippets (not production-ready). The examples are realistic, but highly simplified.

**Notes**:

- This is a Jupyter notebook (formerly iPython notebook, hence the ipynb file extension).
- If using the PowerShell extension for Azure Data Studio as shown in this presentation: it requires the PowerShell extension. It also requires use of .NET (PowerShell) kernel (aka .NET interactive) instead of regular PowerShell kernel in order to use PowerShell Core in the notebook cells. (This requirement was last verified Nov. 2020.) See [this video](https://www.youtube.com/watch?v=W-F0gO7dVOE) on .NET interactive notebooks for more details.
- The Power BI Management module for PowerShell is required to be installed to run most of these scripts:  [https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt](https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt)
- The Data Gateway module is required to be installed to run the last few scripts:  [https://www.powershellgallery.com/packages/DataGateway](https://www.powershellgallery.com/packages/DataGateway)
- Additional information is available in the accompanying presentation materials.

## **Demo Series #1: Power BI Management Module**

## 1a. Check PowerShell version

- The Power BI Management module is supported on Windows PowerShell or PowerShell Core.  
- However, PowerShell Core (7.0+) is required for the Data Gateway module.

In [ ]:
$PSVersionTable

## 1b. Check Power BI Management module version(s)

- This script checks which version(s) of the Power BI Management Module are installed & where.

In [ ]:
Get-Module MicrosoftPowerBIMgmt* -ListAvailable | Sort-Object Version, Name 

## 1c. Check Power BI Management module commands

- The first script lists cmdlets, sorted by name.
- The second script retrieves just the Profile module commands. Note there are several aliases for Connect-PowerBIServiceAccount.

In [ ]:
$Version = '1.0.896'
Get-Command -Module MicrosoftPowerBIMgmt* -Version $Version | Sort-Object Name 

In [ ]:
$Version = '1.0.896'
Get-Command -Module MicrosoftPowerBIMgmt* | Where-Object {$PSItem.Source -eq "MicrosoftPowerBIMgmt.Profile" -and $PSItem.Version -eq $Version} | Sort-Object Source, Name

## 1d. Get help for a cmdlet

- The first script displays the standard help information for the Get-PowerBIWorkspace cmdlet.
- The second script displays example syntax.

In [ ]:
Get-Help Get-PowerBIWorkspace

In [ ]:
Get-Help Get-PowerBIWorkspace -Examples

## <mark>1e. Authenticate to the Power BI Service</mark>

- This script interactively prompts for the user email address and the password to be passed into a credential object.
    
- For simplicity, a domain user is being used for authentication. The domain user must possess Power BI Administrator permissions to execute some of the scripts in this notebook (i.e., the the admin cmdlets & APIs which rely on accessing organization-wide metadata for the entire tenant). For automated / scheduled processes, a better practice is to use an Azure AD application (service principal) -- this became supported for the admin cmdlets & APIs as of Dec 2020.
    
- The Power BI Management Module is used for all authentication throughout all scripts in this notebook (including when APIs are being called because they're being called by the Management Module)..
    
- All scripts below this point in the notebook require authentication. After signing in, the token acquired is active for one hour.

In [1]:
#Prompt for account name:
[string]$DomainUserEmailAddr = Read-Host -Prompt "Input domain user email address"

#Prompt for account password: 
[securestring]$DomainUserPW = Read-Host -Prompt "Input password for $DomainUserEmailAddr" -AsSecureString

#Create temporary credential object: 
[pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($DomainUserEmailAddr, $DomainUserPW)  

#Log into Power BI Service with credential:
Connect-PowerBIServiceAccount -Credential $CredentialObj

Environment : Public


TenantId    : a198ef79-eca1-48c6-b11b-dc15fe119081


ClientId    : sqlchick@sqlchick.com


Password    : ********


## 1f. View workspace info - based on user scope

- The first script returns the first 2 workspaces it finds, using permissions levels of the signed-in user. If -All is not specified, by default it returns the first 100 workspaces.
- The second script returns one workspace, using permissions of the signed-in user.

In [2]:
Get-PowerBIWorkspace -First 2

Id                    : 8999e889-dcdf-4704-96a6-7913d80b4cc0


Name                  : IT Operations Group


IsReadOnly            : False


IsOrphaned            : False


IsOnDedicatedCapacity : False


...

Name                  : Device Telemetry Data


IsReadOnly            : False


IsOrphaned            : False


IsOnDedicatedCapacity : False


CapacityId            : 


In [3]:
Get-PowerBIWorkspace -Name 'Sales Analytics'

Id                    : c7bffcd8-8156-466a-a88f-0785de2c8b13


Name                  : Sales Analytics


IsReadOnly            : False


IsOrphaned            : False


IsOnDedicatedCapacity : False


CapacityId            : 


## 1g. View workspace info - based on organization (admin) scope

- The first script returns one workspace. It requires Power BI admin permissions due to the organization scope (thus this metadata can be returned whether or not the administrator possesses workspace permissions).
- The second script returns all active workspaces for the organization & reformats certain columns for display. It also requires Power BI admin permissions due to the organization scope.

In [4]:
Get-PowerBIWorkspace -Scope Organization -Name 'Sales Analytics'

Id                    : c7bffcd8-8156-466a-a88f-0785de2c8b13


Name                  : Sales Analytics


Description           : MTD, QTD, YTD sales reporting


Type                  : Workspace


State                 : Active


...

                        Viewer       


                        Admin        


                        Viewer       


                        Viewer       


                        Member       analyst@sqlchick.com


In [5]:
Get-PowerBIWorkspace -Scope Organization | Where-Object State -eq 'Active' | Format-Table Name, Type, IsOnDedicatedCapacity 

Name                                                     Type          IsOnDedicatedCapacity


----                                                     ----          ---------------------


Device Telemetry Data                                    Workspace                     False


Sales Analytics                                          Workspace                     False


Common Data                                              Workspace                     False


...

PersonalWorkspace svcPowerBIAdminAutomation              PersonalGroup                 False


PersonalWorkspace Scottie                                PersonalGroup                 False


IT Operations Group                                      Group                         False


PersonalWorkspace SQLChickPurview                        PersonalGroup                 False


PersonalWorkspace Melissa                                PersonalGroup                 False


## 1h. View workspace artifact info

- The first script returns a list of all dashboards in the tenant. Note that the results do not indicate which workspace the object resides in.
- The second script returns a list of dashboards for a workspace. In this example, the parent workspace info is available in addition to the dashboards info. The results are displayed as a JSON object.
- The third script incorporates the -Include All parameter to retrieve all objects (reports, dashboards, workbooks, dataflows, and datasets) for one workspace.

In [6]:
Get-PowerBIDashboard -Scope Organization | Sort-Object Name 

Id                                   Name                                    IsReadOnly EmbedUrl


--                                   ----                                    ---------- --------


e780ccdc-ae8d-47ab-a215-67e6c66db9d4 [App] Call Center Dashboard                   True https://ap…


4e4ae141-b4f0-43e8-a630-4e758731802a [App] Power BI Premium Capacity Metrics       True https://ap…


19c98387-82da-424d-bf1d-d694fee1d40f [App] Sales KPIs                              True https://ap…


...

6aaf1286-f811-4a56-aec1-1f853d827b0b Retail Analysis Sample                       False https://ap…


f2fd89ba-c64e-42e1-b109-620fcc03c03c Sales Data                                   False https://ap…


c937cee3-1e79-4b6a-9946-d7c3820673f5 Sales KPIs                                   False https://ap…


4db8e11d-49c5-44ad-8c41-9b5044bda34a Southeast Sales Division Dashboard           False https://ap…


aba9ec2e-a55e-4d5d-a0a2-f5c2123cc7da YTD Sales Dashboard                          False https://ap…


In [7]:
Get-PowerBIWorkspace -Scope Organization -Name 'Sales Analytics' -Include Dashboards | ConvertTo-JSON -Depth 10 

{
  "Id": "c7bffcd8-8156-466a-a88f-0785de2c8b13",
  "Name": "Sales Analytics",
  "IsReadOnly": false,
  "IsOnDedicatedCapacity": false,
  "CapacityId": null,
  "Description": "MTD, QTD, YTD sales reporting",
  "Type": "Workspace",
  "State": "Active",
  "IsOrphaned": false,
  "Users": [
    {
      "AccessRight": "Admin",
      "UserPrincipalName": "sqlchick@sqlchick.com",
      "Identifier": "sqlchick@sqlchick.com",
      "PrincipalType": 2
    },
    {
      "AccessRight": "Member",
      "UserPrincipalName": null,
      "Identifier": "250bf29f-3f8d-41bb-824a-da418354423b",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Contributor",
      "UserPrincipalName": null,
      "Identifier": "d10ad716-d446-479e-aa64-6cf6712b761d",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Viewer",
      "UserPrincipalName": null,
      "Identifier": "063b9e42-d164-4abb-a303-0a20656dce3d",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Admin",
      "UserPrincipal

In [8]:
Get-PowerBIWorkspace -Scope Organization -Name 'Sales Analytics' -Include All | ConvertTo-JSON -Depth 10 

{
  "Id": "c7bffcd8-8156-466a-a88f-0785de2c8b13",
  "Name": "Sales Analytics",
  "IsReadOnly": false,
  "IsOnDedicatedCapacity": false,
  "CapacityId": null,
  "Description": "MTD, QTD, YTD sales reporting",
  "Type": "Workspace",
  "State": "Active",
  "IsOrphaned": false,
  "Users": [
    {
      "AccessRight": "Admin",
      "UserPrincipalName": "sqlchick@sqlchick.com",
      "Identifier": "sqlchick@sqlchick.com",
      "PrincipalType": 2
    },
    {
      "AccessRight": "Member",
      "UserPrincipalName": null,
      "Identifier": "250bf29f-3f8d-41bb-824a-da418354423b",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Contributor",
      "UserPrincipalName": null,
      "Identifier": "d10ad716-d446-479e-aa64-6cf6712b761d",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Viewer",
      "UserPrincipalName": null,
      "Identifier": "063b9e42-d164-4abb-a303-0a20656dce3d",
      "PrincipalType": 1
    },
    {
      "AccessRight": "Admin",
      "UserPrincipal

## 1i. Export workspace inventory

- This script extracts all objects for all workspace, and exports them to one JSON file, which is considered a snapshot at that point in time.
- This approach is useful for larger tenant which have a lot of data, as it minimizes the # of API calls to be made. The limit is 200 API calls per hour.
- This technique is useful when you want to store a snapshot of workspace inventory on a regular basis, and/or your want to see changes over time with point-in-time snapshots. 
- The JSON format is used because, over time, new data elements will be introduced and a flexible schema is necessary to accomodate that.

In [9]:
[string]$ExportFileLocation = 'C:\Demos\Demo-Output'
[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")  #Snapshot date when data was extracted
[string]$ExportFileName = 'AllWorkspaceObjects-FromCmdlet-' + $DateTimeFileWrittenUTCLabel + '.json' #FromCmdlet in the name since the next example does the same thing using the API directly

Get-PowerBIWorkspace -Scope Organization -Include All | ConvertTo-JSON -Depth 10 | Out-File "$ExportFileLocation\$ExportFileName"

Write-Verbose "File written: $ExportFileName" -Verbose 

VERBOSE: File written: AllWorkspaceObjects-FromCmdlet-202102021537.json


## **Demo Series #2: Power BI REST APIs**

## 2a. View workspace artifact info for one workspace

- This script calls the admin groups API.
- It uses the $expand parameter to retrieve metadata about all types of artifacts for the workspace. 
- It includes a filter parameter on workspace name.

In [ ]:
[int32]$NbrOfRecordsToFetchPerBatch = 1000 
[int32]$NbrOfRecordsToSkip = 0 
[string]$WorkspaceName = '''Sales Data'''

[string]$URL = "https://api.powerbi.com/v1.0/myorg/admin/Groups?%24top=$NbrOfRecordsToFetchPerBatch&%24skip=$NbrOfRecordsToSkip&%24expand=datasets,dataflows,reports,dashboards,workbooks,users&%24filter=name eq $WorkspaceName"

Invoke-PowerBIRestMethod -Url $URL -Method GET

## 2b. Export workspace inventory - using REST API

- This script accomplishes the same thing as the earlier 'Export workspace inventory' script; this one just uses the REST API instead.
- It calls the Power BI Groups REST API via the 'Invoke-PowerBIRestMethod' cmdlet from the Power BI Management Module. Therefore, the Power BI Management Module is still acting like a 'wrapper' around the API call.
- This technique allows us to use authentication with the Power BI Management Module, which is simpler than OAuth authentication.
- The $expand parameter specifies which objects are included.
- The %24 shown in the URL is because the $ sign is used by PowerShell for variables & parameters. For this reason, the $ sign is replaced in PowerShell scripts with the %24.

In [10]:
[int32]$NbrOfRecordsToFetchPerBatch = 1000 
[int32]$NbrOfRecordsToSkip = 0 

[string]$ExportFileLocation = 'C:\Demos\Demo-Output'
[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")  #Snapshot date when data was extracted
[string]$ExportFileName = 'AllWorkspaceObjects-FromAPI-' + $DateTimeFileWrittenUTCLabel + '.json' #FromAPI in the name since the previous example did the same thing using the cmdlet

[string]$URL = "https://api.powerbi.com/v1.0/myorg/admin/Groups?%24top=$NbrOfRecordsToFetchPerBatch&%24skip=$NbrOfRecordsToSkip&%24expand=datasets,dataflows,reports,dashboards,workbooks,users"

Invoke-PowerBIRestMethod -Url $URL -Method GET | Out-File "$ExportFileLocation\$ExportFileName"

Write-Verbose "File written: $ExportFileName" -Verbose 

VERBOSE: File written: AllWorkspaceObjects-FromAPI-202102021550.json


## 2c. View apps info

- This script returns apps published for the entire tenant.
- It converts the JSON results into a PowerShell object for viewing.

In [ ]:
$URL = "https://api.powerbi.com/v1.0/myorg/apps"
$AppsInfo = Invoke-PowerBIRestMethod -Url $URL -Method GET
$AppsObject = $AppsInfo | ConvertFrom-Json
$AppsObject.value 

## 2d. View dataset and report info for PBIX files in a workspace

- This script displays the dataset and report metadata for each imported PBIX file in a workspace, such as when the PBIX file was last updated. (The file update date is separate from when it was refreshed--see the next example below for getting refresh date.)
- Step 1 gets the workspace ID based on the name entered in the variable. The API requires single quotes to be sent around the workspace name, which is why the $WorkspaceName variable is constructed the way it is. This is an admin API which requires administrator permissions.
- Step 2 gets the PBIX file info. The imports API does not extend to related objects or lineage beyond a single PBIX.

In [ ]:
[int32]$NbrOfRecordsToFetchPerBatch = 1000 
[int32]$NbrOfRecordsToSkip = 0 
[string]$WorkspaceName = '''Sales Data'''

#Step 1:
[string]$URL = "https://api.powerbi.com/v1.0/myorg/admin/Groups?%24top=$NbrOfRecordsToFetchPerBatch&%24skip=$NbrOfRecordsToSkip&%24filter=name eq $WorkspaceName"
$Workspace = Invoke-PowerBIRestMethod -Url $URL -Method GET
$WorkspaceObject = $Workspace | ConvertFrom-Json 
$WorkspaceID = $WorkspaceObject.value.id 

#Step 2:
$ImportFileURL = "https://api.powerbi.com/v1.0/myorg/groups/$WorkspaceID/imports"
$ImportInfo = Invoke-PowerBIRestMethod -Url $ImportFileURL -Method GET
$ImportObject = $ImportInfo | ConvertFrom-Json
$ImportObject.value 

## 2e. View dataset last refresh date

- This script displays the last refresh date for a dataset, as well as other information such as how it was refreshed and the status.
- Step 1 gets the workspace info because the ID will be needed in step 2.
- Step 2 gets the datasets in the workspace which was specified. Each time the loop runs, the results are added to an array which is referenced in step 3.
- Step 3 calls the refresh history API to retrieve the latest (top 1) refresh for each dataset in the workspace which was specified. It is converted from JSON into a PowerShell object for easier viewing in the console. Additional properties for the DatasetID and DatasetName are added to the resultset to help identify the lineage of the refresh information. Also shown is aliasing the ID column so its meaning is more clear. 

This technique is shown to be able to illustrate the logic. However, it does issue quite a few API calls in the looping structure. In large environments with a lot of artifacts, a different technique should be used (such as the workspace with the $expand property shown earlier).

In [ ]:
$WorkspaceName = 'Sales Data'
$NbrOfRefreshes = 1 

#Step 1:
$Workspaces = Get-PowerBIWorkspace -Scope Organization -Name $WorkspaceName | Select-Object Id 

#Step 2:
$Datasets = @()
    foreach ($Workspace in $Workspaces)  
    {
        $Dataset = Get-PowerBIDataset -Scope Organization -WorkspaceId $Workspace.Id | Select-Object Id, Name
        $Datasets += $Dataset
    }

#Step 3:
foreach ($Dataset in $Datasets)
    { 
        $DatasetID = $Dataset.Id
        $URL = "https://api.powerbi.com/v1.0/myorg/datasets/$DatasetID/refreshes?%24top=$NbrOfRefreshes"
        $RefreshInfo = Invoke-PowerBIRestMethod -Url $URL -Method GET
        $RefreshObject = $RefreshInfo | ConvertFrom-Json 
        $DatasetRefreshInfo = $RefreshObject.value
        $DatasetRefreshInfo | Add-Member -MemberType 'NoteProperty' -Name 'DatasetID' -Value $Dataset.Id -Force 
        $DatasetRefreshInfo | Add-Member -MemberType 'NoteProperty' -Name 'DatasetName' -Value $Dataset.Name -Force 
        $DatasetRefreshInfo | Add-Member -MemberType 'AliasProperty' -Name 'RefreshID'  -Value id
        $DatasetRefreshInfo 
    }

## **Demo Series #3: Power BI Activity Events**

## 3a. View one type of activity events for one day

View data using cmdlet from the Power BI Management Module: Get-PowerBIActivityEvent.

- This script displays the "ShareReport" events, for one day. 
- It interactively prompts for the date. Data up to 30 days in the past may be retrieved.
- All activity events are based on UTC datetime.

In [13]:
[string]$DateToExtractLabel = Read-Host -Prompt "Input date to extract in yyyy-mm-dd format" #Maximum supported is 30 days back

Get-PowerBIActivityEvent -StartDateTime ($DateToExtractLabel+'T00:00:00.000') -EndDateTime ($DateToExtractLabel+'T23:59:59.999') -ActivityType 'ShareReport'

[
  {
    "Id": "22093aa3-b7dd-4914-9404-6a6324745572",
    "RecordType": 20,
    "CreationTime": "2021-01-27T23:47:45Z",
    "Operation": "ShareReport",
    "OrganizationId": "a198ef79-eca1-48c6-b11b-dc15fe119081",
    "UserType": 0,
    "UserKey": "10033FFF93C8217B",
    "Workload": "PowerBI",
    "UserId": "sqlchick@sqlchick.com",
    "ClientIP": "69.132.26.0",
    "UserAgent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0",
    "Activity": "ShareReport",
    "ItemName": "Product Sales Analysis",
    "WorkSpaceName": "Sales Analytics",
    "SharingInformation": [
      {
        "RecipientEmail": "analyst@sqlchick.com",
        "ResharePermission": "Read"
      }
    ],
    "WorkspaceId": "c7bffcd8-8156-466a-a88f-0785de2c8b13",
    "ObjectId": "Product Sales Analysis",
    "Datasets": [
      {
        "DatasetId": "1fc46e65-b402-41b4-976d-69cfb70bb16c",
        "DatasetName": "Sales Data"
      }
    ],
    "ArtifactId": "c8f2ca7a-232b-4bb5-adf3-7f

## 3b. Export activity events for a range of days

- This script exports raw data for all Power BI events.
- It exports one JSON file per day (based on UTC time), which is useful when you want to store raw data over time. 
- The range of time is specified with the first variable. It counts backwards from yesterday to determine how many days it should loop through.
- A maximum of one day is allowed to be extracted at once, which is why a loop is introduced per day. 
- The JSON format is used because, over time, new data elements will be introduced and a flexible schema is necessary to accomodate that. It is a best practice to store the raw data as-is, in an immutable location when possible, so it can be used for auditing when necessary.

In [14]:
[int32]$NbrDaysDaysToExtract = 7  #Maximum is 30 days back
[string]$ExportFileLocation = 'C:\Demos\Demo-Output'

[datetime]$YesterdayUTC = (([datetime]::Today.ToUniversalTime()).Date).AddDays(-1) #Begin with yesterday, rather than today, to ensure full day results are obtained
[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")

#Loop through each of the days to be extracted (<Initilize> ; <Condition> ; <Repeat>):
for($Loop=0 ; $Loop -lt $NbrDaysDaysToExtract ; $Loop++)
{
    [datetime]$DateToExtractUTC=$YesterdayUTC.AddDays(-$Loop).ToString("yyyy-MM-dd")

    [string]$DateToExtractLabel=$DateToExtractUTC.ToString("yyyy-MM-dd")
    
    [string]$ExportFileName = 'AllActivityEvents-' + ($DateToExtractLabel -replace '-', '') + '-' + $DateTimeFileWrittenUTCLabel + '.json' 

    [psobject]$Events=Get-PowerBIActivityEvent -StartDateTime ($DateToExtractLabel+'T00:00:00.000') -EndDateTime ($DateToExtractLabel+'T23:59:59.999')
  
    $Events | Out-File "$ExportFileLocation\$ExportFileName"

    Write-Verbose "File written: $ExportFileName" -Verbose 
}

VERBOSE: File written: AllActivityEvents-20210201-202102021601.json


VERBOSE: File written: AllActivityEvents-20210131-202102021601.json


VERBOSE: File written: AllActivityEvents-20210130-202102021601.json


VERBOSE: File written: AllActivityEvents-20210129-202102021601.json


VERBOSE: File written: AllActivityEvents-20210128-202102021601.json


VERBOSE: File written: AllActivityEvents-20210127-202102021601.json


VERBOSE: File written: AllActivityEvents-20210126-202102021601.json


## **Demo Series #4: Data Gateway Module**

## 4a. Check Data Gateway module version(s)

- This script checks which version(s) of the Data Gateway Module are installed. Unlike the Power BI Management Module, the Data Gateway Module requires PowerShell Core (7.0+).

In [ ]:
Get-Module DataGateway* -ListAvailable | Sort-Object Version, Name 

## 4b. Check Data Gateway module commands

- The first script lists all cmdlets & aliases, for all versions.
- The second script retrieves just the Profile module commands. Similar to the Power BI Mgmt Module, there are several aliases for Connect-DataGatewayServiceAccount.

In [ ]:
Get-Command -Module DataGateway*

In [ ]:
Get-Command -Module DataGateway* | Where-Object {$PSItem.Source -eq "DataGateway.Profile"} | Sort-Object Source, Name

## <mark>4c. Authenticate to the Data Gateway service</mark>

- Note this is a \*\*separate\*\* login cmdlet from the Power BI Management Module. 
- This interactive login will complete in your default web browser.

In [ ]:
Connect-DataGatewayServiceAccount  

## 4d. View gateway cluster info

- This script uses the data gateway module to view metadata about each gateway cluster.

In [ ]:
Get-DataGatewayCluster -Scope Organization

## 4e. View gateway data source info

- This script displays each data source registered to the gateway cluster, including the type. Server name and database name are included in the connection details.

In [ ]:
$GWClusters = Get-DataGatewayCluster -Scope Organization

foreach ($GWCluster in $GWClusters)  
{
    Get-DataGatewayClusterDatasource -GatewayClusterId $GWCluster.Id
}

## 4f. View data source users for each gateway data source

- This script displays the users which have been added to each data source.
- Step 1 gets the gateway cluster info. It passes the cluster ID to step 2.
- Step 2 retrieves each data source for the cluster. It passes the ID, name, and cluster ID to step 3.
- Step 3 retrieves each user assigned to the data source. Additional properties (Cluster ID, Data Source Name, and Data Source ID) are added to the resultset to help identify the lineage of the user information.

In [ ]:
#Step 1:
$GWClusters = Get-DataGatewayCluster -Scope Organization

#Step 2:
$GWClusterSources = @()
foreach ($GWCluster in $GWClusters)  
{
    $GWClusterSource = Get-DataGatewayClusterDatasource -GatewayClusterId $GWCluster.Id | Select-Object Id, DatasourceName, ClusterId
    $GWClusterSources += $GWClusterSource
}

#Step 3:
foreach ($GWClusterSource in $GWClusterSources)
{ 
    $Users = Get-DataGatewayClusterDatasourceUser -GatewayClusterId $GWClusterSource.ClusterId -GatewayClusterDatasourceId $GWClusterSource.Id
    $Users | Add-Member -MemberType 'NoteProperty' -Name 'ClusterID' -Value $GWClusterSource.ClusterId -Force 
    $Users | Add-Member -MemberType 'NoteProperty' -Name 'DataSourceName' -Value $GWClusterSource.DatasourceName -Force
    $Users | Add-Member -MemberType 'NoteProperty' -Name 'DataSourceID' -Value $GWClusterSource.Id -Force
    $Users 
}

## 4g. Disconnect from the Power BI service & Data Gateway service

These cmdlets are useful to place at the end of automation scripts, as a matter of good housekeeping.

In [ ]:
Disconnect-PowerBIServiceAccount

In [ ]:
Disconnect-DataGatewayServiceAccount